In [3]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [4]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr = data_collection(robot=panda, N=config.N_train)
J_ts, P_ts = data_collection(robot=panda, N=config.N_test)
F = posture_feature_extraction(J_tr)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()
# knn = get_knn(P_tr)
knn = load_pickle(file_path=config.path_knn)

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [5]:
mu = torch.zeros(size=(config.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [6]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.31)
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=iflow, knn=knn, K=NUM_SAMPLES, print_report=True)

       position errors (m)
count        100000.000000
mean              0.002860
std               0.007244
min               0.000018
25%               0.001274
50%               0.002025
75%               0.003084
max               0.416129
average inference time (of 100 P): 0.05 sec.


,position errors (m)
0,0.002492
1,0.001688
2,0.001801
3,0.000903
4,0.002171
...,...
99995,0.007103
99996,0.005881
99997,0.005787
99998,0.003426


In [7]:
nflow = get_nflow_model(flow=flow)
inference(robot=panda, P_inf=P_ts[:NUM_DATA], F=F, solver=nflow, knn=knn, K=NUM_SAMPLES, print_report=True)

       position errors (m)
count        100000.000000
mean              0.003033
std               0.008861
min               0.000025
25%               0.001306
50%               0.002073
75%               0.003150
max               0.452436
       inference time (sec) of 1000 samples
count                            100.000000
mean                               0.080200
std                                0.001407
min                                0.080000
25%                                0.080000
50%                                0.080000
75%                                0.080000
max                                0.090000


In [12]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

config.py: Using device 'cuda:0'


In [13]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

set_seed() - random int:  44
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
ndim_tot=7
dim_cond=8
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link0.stl (595 verts, 200 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link1.stl (887 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link2.stl (889 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link3.stl (900 verts, 300 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/collision/link4.stl (9

In [8]:
def get_target_poses(P_ts):
    quaternions = np.zeros((len(P_ts), 4))
    quaternions[:, 0] = np.random.randn() * 2e-2 + 1
    target_poses = np.column_stack((P_ts, quaternions))
    return target_poses

In [10]:
target_poses = get_target_poses(P_ts[:NUM_DATA])
target_poses.shape

(100, 7)

In [15]:
# -> unrefined solutions
l2_errs = np.zeros((len(target_poses), NUM_SAMPLES))
time_diffs = np.zeros((len(target_poses), NUM_SAMPLES))

for i, target_pose in enumerate(target_poses):
    iksols, l2_err, _, _, _, dt = ik_solver.solve(
        target_pose, 
        n=NUM_SAMPLES,
        refine_solutions=False, 
        return_detailed=True)

    iksols = iksols.detach().cpu().numpy()
    dt = np.zeros_like(l2_err) + dt
    
    l2_errs[i] = l2_err
    time_diffs[i] = dt

l2_errs = l2_errs.reshape((-1))
time_diffs = time_diffs.reshape((-1))

    
df = pd.DataFrame(
    data=np.column_stack((l2_errs, time_diffs)),
    columns=["l2_err", "time_diff"],
)

Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() with pytorch directly on the cpu/gpu
Heads up: It may be faster to run solution_pose_errors() wit

In [16]:
df.describe()

,l2_err,time_diff
count,100000.000000,100000.000000
mean,0.061731,0.195984
std,0.173679,0.022551
min,0.000122,0.162961
25%,0.003596,0.186114
50%,0.005818,0.193867
75%,0.027328,0.203522
max,1.525950,0.352305
